# Fleming Pipeline - ipython notebook version

Note on astrometry.net keys:
[Go to the website](http://nova.astrometry.net/api_help) and sign in and follow the instructions.

Need to add your own key to `~/.astroquery/config/astroquery.cfg` and `pipeline/astrometry_api_key.txt`

In [ ]:
## Imports
import Reducer
import ShiftFinder
import FluxFinder
import DataAnalyser
import Constants
import Utilities
import MovingObjectFinder
import StreakFinder
import Cataloguer

from datetime import datetime
import os
import importlib

#from astroquery import log
#log.setLevel("TRACE")


In [ ]:
## Reload imports (if files changed, only used for testing)
ShiftFinder = importlib.reload(ShiftFinder)
FluxFinder = importlib.reload(FluxFinder)
DataAnalyser = importlib.reload(DataAnalyser)
Utilities = importlib.reload(Utilities)
Cataloguer = importlib.reload(Cataloguer)
Constants = importlib.reload(Constants)


In [ ]:
# REMOVE ME WHEN NOT USING DARK MODE
import matplotlib as mpl
COLOR = "white"
#COLOR = "black"
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

In [ ]:
config = Constants.Config(
    image_subdir = "../images/0121",
    image_prefix = "l137_0",
    has_sets = True,
    set_size = 50,
    n_sets = 7
)

In [ ]:
#r = Reducer.Reducer(Constants.folder,"No filter",Constants.file_name, 9, 50)
#r.reduce(True)
# # #set_size, n_sets = r.get_set_info()

In [ ]:
c = Cataloguer.Cataloguer(config)
catalogue_image = os.path.join(config.image_dir,
        config.image_format_str.format(1, 1))
#print(catalogue_image)
c.catalogue(catalogue_image)
Utilities.print_job("cataloguing images")

In [ ]:
sf = ShiftFinder.ShiftFinder(config)
sf.get_all_shifts()
Utilities.print_job("finding shifts")

In [ ]:
ff = FluxFinder.FluxFinder(config)
ff.find_all_fluxes()
ff.make_light_curves()
Utilities.print_job("making light curves")

In [ ]:
r = None
c = None
sf = None

da = DataAnalyser.DataAnalyser(config)

da.get_means_and_stds(False)
da.get_variables(False)
#da.output_results() #this should not be here - make it just create results folder with separate method
da.plot_means_and_stds("_adjusted")
ids = da.get_ids_for_avg()

ff.make_avg_curve(ids)
ff.divide_by_average()
avg_fname = "{}_avg{}".format(config.image_prefix, config.standard_file_extension)
ff.plot_light_curve(None, os.path.join(config.workspace_dir, avg_fname), True)
Utilities.print_job("adjusting light curves")

In [ ]:
da.get_means_and_stds(True)
da.get_variables(True)
da.plot_means_and_stds(True)
da.output_results()
da.create_thumbnails(ff)

Utilities.print_job("everything")